## Certified AI Practitioner Week 04 Call 02 - From 🤗 to Chatbot: Open-Source AI in Production

## Learning Objectives

- Understand what conversational AI is and how `DialoGPT` enables it  
- Load and use a Hugging Face chatbot model locally with the `transformers` library  
- Maintain conversation history to support multi-turn dialogue  
- Identify trade-offs between different deployment architectures (e.g., Lex vs SageMaker)  
- Discuss real-world considerations for chatbot safety, scalability, and cost  
- Connect local development with what production-ready chatbot systems look like

## Example Model: `microsoft/DialoGPT-medium`
### Explore the Model on Hugging Face  

`DialoGPT` is a conversational model fine-tuned from GPT-2 by Microsoft for multi-turn dialogue. It’s optimized to generate human-like responses in chat-style applications.

**Model Card:**  
→ [View on Hugging Face](https://huggingface.co/microsoft/DialoGPT-medium)


### Conversation Behavior

This model expects:
- **Sequential prompts** where each response depends on prior inputs  
- **Manual context handling** (i.e., feed the entire conversation history for coherent replies)

### Try It in the Browser + Explore Spaces

On the model page, scroll down to the **"Hosted inference API"** to test the model live (if available).  
You’ll also find a list of community-built **Spaces** that use this model — from basic chatbot UIs to creative applications.

> Some Spaces may be experimental or contain NSFW content — see the note below on community moderation.

---

> **⚠️ Note on Hugging Face Community Content**  
> Hugging Face hosts open-source models and apps ("Spaces") contributed by the global ML community. While this enables rapid innovation and access to cutting-edge tools, it also means some content may be unmoderated or NSFW.  
>  
> In production environments, it's critical to:
> - Review model cards and licenses  
> - Validate model behavior before deploying  
> - Apply content moderation and filtering when necessary  
> - Prefer verified models for enterprise use cases

## Set Up Your Environment

Before using the DialoGPT model, you'll need to install the required libraries and import necessary modules.

```python
# Install Hugging Face transformers if not already installed
!pip install -q transformers

# Import core libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
```

> **Note:** DialoGPT is a PyTorch-based model, so we’ll be using the PyTorch version of Hugging Face’s `transformers` library.

## Load the Chatbot Model

We'll load the `microsoft/DialoGPT-medium` model along with its tokenizer. This model is optimized for conversational, multi-turn dialogue.

```python
# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
```

> **Note:** This loads both the tokenizer (which converts text to tokens) and the model (which generates responses) from Hugging Face’s model hub.

## Build a Local Conversational Loop

Now let's build a simple interactive chatbot loop using `DialoGPT`.  
This script lets you chat with the model in real time, maintaining conversation history.

```python
# Initialize conversation history
chat_history_ids = None
attention_mask = None

print("Chatbot is ready! Type 'quit' to exit or 'reset' to clear conversation history.")
for step in range(5):  # Adjust as needed
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Goodbye!")
        break
    if user_input.lower() == "reset":
        chat_history_ids = None
        attention_mask = None
        print("Chat history cleared.")
        continue

    # Encode input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    new_attention_mask = torch.ones_like(new_input_ids)

    # Combine with chat history
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        attention_mask = torch.cat([attention_mask, new_attention_mask], dim=-1)
    else:
        bot_input_ids = new_input_ids
        attention_mask = new_attention_mask

    # Generate response with attention mask
    output_ids = model.generate(
        bot_input_ids,
        attention_mask=attention_mask,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95
    )

    # Update history
    chat_history_ids = output_ids
    attention_mask = torch.ones_like(chat_history_ids)

    # Decode and print reply
    response = tokenizer.decode(output_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Bot: {response}")
```

> This loop maintains conversational context by appending each input to a running history. The model sees the entire dialogue, making its replies more coherent.


## Clean Up the Conversation History (Optional)

In real-world chatbot applications, you may want to reset or manage the conversation history based on session limits, user actions, or context drift.

You can clear the context at any time by resetting `chat_history_ids`:

```python
# Reset the chat history (e.g., after N turns or when the user types 'reset')
chat_history_ids = None
print("Chat history cleared.")
```
> Tip: In production, you might also limit the total token count or summarize long histories to reduce input size and cost.


## Bonus: Where This Fits in Production

Building a chatbot with `DialoGPT` locally is a great proof of concept — but how do real teams scale this?

In production, this model could be:

- **Packaged in a Docker container**
- **Hosted on SageMaker, ECS, or Lambda with EFS**
- **Accessed via API Gateway or a chatbot UI**
- **Integrated with a frontend (like a website or support chat)**

You’d also want to:

- Add authentication and throttling  
- Log requests/responses for review  
- Add safety filters and tone moderation  
- Control cost by scaling down idle services

> Remember: A working chatbot ≠ a production-ready product. Think about user safety, monitoring, and cost control from day one.


## Cost-Aware Architecture Decisions

When deploying chatbots, it’s easy to overbuild for scale that may never come. In most real-world use cases, **simplicity saves money** — especially during early development or internal pilots.

### Option A: Hugging Face + SageMaker
- Pay for **instance time** (e.g., `ml.t2.medium`, `ml.g5.xlarge`)
- Good for custom models or full control over inference
- Requires MLOps setup: IAM, monitoring, scaling, shutdowns

### Option B: Amazon Lex (NLU-based Chatbot)
- Pay **only for usage** (per message)
- Easy UI builder, built-in slot filling + Lambda hooks
- Best for forms, menus, support bots with clear flows

---

### Comparing Costs

| Architecture      | Best For                    | Cost Model          | Example |
|-------------------|-----------------------------|----------------------|---------|
| SageMaker + 🤗     | Custom LLMs, GenAI chat     | Per hour (instance) | ~$0.05–$1/hr |
| Amazon Lex        | Menus, forms, call centers  | Per request (usage) | ~$4/month for low usage |

> Most startups and internal teams don’t get millions of chats per day.  
> Start small, monitor usage, and only scale when needed.


## Wrap-Up & Takeaways 

## In This Notebook

In this notebook, you explored how to build and test a conversational AI model locally using Hugging Face’s `DialoGPT`. You:

- Explored the `DialoGPT-medium` model on Hugging Face and reviewed key metadata  
- Loaded the model and tokenizer locally with the `transformers` library  
- Built a conversational loop that maintains chat history  
- Learned how to reset context and handle multi-turn dialogue  
- Discussed production architecture options and cost-aware deployment strategies  

---

## This Workflow Reflects What Real ML Teams Do in Production

- Evaluate open-source models before committing to deployment  
- Test model behavior locally using realistic user inputs  
- Consider security, moderation, and cost early in the design process  
- Build APIs and chat interfaces around pretrained models  
- Choose between managed services (Lex) and custom hosting (SageMaker) based on scale and use case
